#### 针对官方给出的adaptVQE实现

In [22]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureMoleculeDriver, ElectronicStructureDriverType
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer
import warnings
from qiskit_nature.circuit.library import HartreeFock,UCCSD
from qiskit import Aer
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 
warnings.filterwarnings("ignore")


In [23]:
molecule = Molecule(
    # coordinates are given in Angstrom
    geometry=[
        ["O", [0.0, 0.0, 0.0]],
        ["H", [0.758602, 0.0, 0.504284]],
        ["H", [0.758602, 0.0, -0.504284]]
    ],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)

driver = ElectronicStructureMoleculeDriver(
    molecule=molecule,
    basis="sto3g",
    driver_type=ElectronicStructureDriverType.PYSCF,
)

properties = driver.run()
converter = QubitConverter(JordanWignerMapper())

# Define the active space around the Fermi level 
# (selected automatically around the HOMO and LUMO, ordered by energy)
transformer = ActiveSpaceTransformer(
    num_electrons=2, #how many electrons we have in our active space
    num_molecular_orbitals=2, #how many orbitals we have in our active space
)

problem = ElectronicStructureProblem(driver, [transformer])
problem.second_q_ops()
print('粒子数：{0} 轨道数:{1}'.format(problem.num_particles,problem.num_spin_orbitals))


粒子数：(1, 1) 轨道数:4


In [24]:
from qiskit_nature.second_q.drivers import PySCFDriver
driver2 = PySCFDriver(atom="H 0 0 0; H 0 0 0.735", basis="sto-3g")
problem2 = driver.run()

In [12]:
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter
converter = QubitConverter(JordanWignerMapper())
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
ansatz = UCCSD()
ansatz.num_particles = problem.num_particles
ansatz.num_spatial_orbitals = problem.num_spin_orbitals//2
ansatz.qubit_converter = converter
initial_state = HartreeFock()
initial_state.num_particles = problem.num_particles
initial_state.num_spatial_orbitals = problem.num_spin_orbitals//2
ansatz.qubit_converter = converter
ansatz.initial_state = initial_state

In [18]:
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
vqe = VQE(Estimator(), ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters)

In [19]:
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
adapt_vqe = AdaptVQE(vqe,max_iterations=100,threshold=1e-3)
adapt_vqe.supports_aux_operators = lambda: True  # temporary fix

In [20]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
solver = GroundStateEigensolver(converter, adapt_vqe)
result = solver.solve(problem)

ValueError: too many values to unpack (expected 2)

{'ParticleNumber': FermionicOp([('+_0 -_0', (1+0j)), ('+_1 -_1', (1+0j)), ('+_2 -_2', (1+0j)), ('+_3 -_3', (1+0j)), ('+_4 -_4', (1+0j)), ('+_5 -_5', (1+0j)), ('+_6 -_6', (1+0j)), ('+_7 -_7', (1+0j)), ('+_8 -_8', (1+0j)), ('+_9...)], register_length=14, display_format='sparse'),
 'ElectronicEnergy': FermionicOp([('+_0 -_0', -32.75957523255523), ('+_0 -_1', 0.5358761109404167), ('+_0 -_3', 0.3118348753981393), ('+_0 -_5', 0.26643810499822856), ('+_1 -_0', 0.5358761109404171), ('+_1 -_1', -7.7425796924270...)], register_length=14, display_format='sparse'),
 'DipoleMomentX': FermionicOp([('+_0 -_0', 0.0005561413359251467), ('+_0 -_1', 0.012318293053490512), ('+_0 -_3', -0.04175866877728066), ('+_0 -_5', 0.052632450626007014), ('+_1 -_0', 0.01231829305349051), ('+_1 -_1', 0.53296...)], register_length=14, display_format='sparse'),
 'DipoleMomentY': FermionicOp([('+_0 -_4', 0.06387795069214892), ('+_1 -_4', 0.6005670115874404), ('+_3 -_4', 0.23400649289105202), ('+_4 -_0', 0.0638779506921489